# Importation des bibliothèques nécessaires

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from sklearn import svm
from sklearn import metrics
from sklearn import linear_model
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

# Chargement des données

In [ ]:
data_mnist=tf.keras.datasets.mnist
(X_train,Y_train),(X_test,Y_test)=data_mnist.load_data();

In [ ]:
def plot_mnist_images(images, labels, num_images=20):
    num_cols = 10
    num_rows = num_images // num_cols + (1 if num_images % num_cols != 0 else 0)
    
    plt.figure(figsize=(10, num_rows * 2))
    for i in range(num_images):
        plt.subplot(num_rows, num_cols, i 
                    + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(labels[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()
plot_mnist_images(X_train, Y_train, num_images=30)

In [ ]:
print("Taille des données d'entraînement :", X_train.shape, Y_train.shape)
print("Taille des données de test :", X_test.shape, Y_test.shape)
print(Y_train)
print(X_train)
print(Y_test)
print(X_test)



# Redimensionnement et Normalisation des Données MNIST


In [ ]:

# Aplatissement des images (conversion 28x28 en 784)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Normalisation des données (mise à l'échelle entre 0 et 1)
X_train = X_train / 255.0
X_test = X_test / 255.0
print(X_train.shape)
print(X_test.shape)

# Visualisation des données d'entraînement

In [ ]:
def drawImg(sample):               
    img=sample.reshape((28,28))    
    plt.imshow(img,cmap='gray')
    plt.show()
    
drawImg(X_train[333])
print(Y_train[333])

# Implémentation de KNN

In [ ]:
def euclidean_distance(x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))

def knn(X,Y,query_point,k=5 ):
    vals=[]
    m=X_train.shape[0]
    
    for i in range(m):
        d=euclidean_distance(query_point,X[i])
        vals.append((d,Y[i])) 
            
    vals=sorted(vals)
  
    vals=vals[:k]
    
    vals=np.array(vals)
    #print(vals)
    new_vals=np.unique(vals[:,1],return_counts=True)
    index=new_vals[1].argmax()
    pred=new_vals[0][index]
    
    return pred

In [ ]:
X_test1=X_test[:50]   # Taking the first 50 rows from X_test
Y_test1=Y_test[:50]   # Taking the first 50 rows from Y_test

# Prédictions des données

In [ ]:
for test in X_test1:
    drawImg(test)
    print("Label:", knn(X_train, Y_train, test))

# L'analyse de la performance du modèle KNN

In [ ]:

def predict_knn(X_train, Y_train, X_test, k=5):
    predictions = []
    for test_point in X_test:
        pred_label = knn(X_train, Y_train, test_point, k)
        predictions.append(pred_label)
    return np.array(predictions)


In [ ]:

y_pred = predict_knn(X_train, Y_train, X_test1, k=5)

In [ ]:

# Calcul de l'accuracy
accuracy = accuracy_score(Y_test1, y_pred)
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
# Affichage de la matrice de confusion
conf_matrix1 = confusion_matrix(Y_test1, y_pred)
print("Confusion Matrix:")
print(conf_matrix1)


In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix1, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
# Rapport de classification
class_report = classification_report(Y_test1, y_pred)
print("Classification Report:")
print(class_report)


# Implémentation de CNN

In [ ]:
image_size = 28
X_trainR = np.array(X_train).reshape(-1,image_size,image_size,1)
X_testR = np.array(X_test).reshape(-1,image_size,image_size ,1)
print("training samples dimension ",X_trainR.shape)
print("testing samples dimension ",X_testR.shape)



In [ ]:
# Création d'un réseau de neurones
model= Sequential()
## 1er Convolution layers 
model.add(Conv2D(64,(3,3), input_shape=X_trainR.shape[1:])) 
model.add(Activation("relu"))## Fonction d'activation pour le rendre non linéaire
model.add(MaxPooling2D(pool_size=(2,2)))## maxpooling
## 2rd Convolution layer
model.add(Conv2D(64,(3,3),input_shape=X_trainR.shape[1:])) 
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# 3rd Convolution layer 
model.add(Conv2D(64,(3,3),input_shape=X_trainR.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Fully connected layer 1  20*20 =400
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

#Fully connected layer 2
model.add(Dense(32))
model.add(Activation("relu"))

#Dernière couche entièrement connectée : la sortie doit être égale au nombre de classes, soit 10 (0 à 9).
model.add(Dense(10))
model.add(Activation("softmax"))### La fonction d'activation est changée en softmax.


In [ ]:
model.summary()

In [ ]:
print("totale train Samples is :",len(X_trainR))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#training my model 
model.fit(X_trainR, Y_train, epochs=5, validation_split=0.3)


In [ ]:
test_loss, test_accu = model.evaluate(X_testR,Y_test)
print("test loss on 10000 test samples is :",test_loss)
print ("test accuracy on 10000 samples is :",test_accu)

# Prédictions des données

In [ ]:
predictions2=model.predict(X_testR)

In [ ]:
print(predictions2)

In [ ]:
def plot_images_with_labels(image1, label1, image2, label2):
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(image1, cmap='gray')
    axes[0].axis('off')
    axes[0].set_title('Label: {}'.format(label1))
    axes[1].imshow(image2, cmap='gray')
    axes[1].axis('off')
    axes[1].set_title('Label: {}'.format(label2))
    plt.show()

image1 = X_testR[500]
label1 = np.argmax(predictions2[500])
image2 = X_testR[100]
label2 = np.argmax(predictions2[100])
plot_images_with_labels(image1, label1, image2, label2)

# L'analyse de la performance du modèle CNN

In [ ]:
# Convertir les prédictions en classes
Y_pred_classes = np.argmax(predictions2, axis=1)

# Calculer la matrice de confusion
conf_matrix2 = confusion_matrix(Y_test, Y_pred_classes)

# Tracer la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix2, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
print(conf_matrix2)

In [ ]:
# measure accuracy
metrics.accuracy_score(y_true=Y_test, y_pred=predictions2)
# class-wise accuracy
class_wise = metrics.classification_report(y_true=Y_test, y_pred=predictions2)
print(class_wise)

# Implémentation de SVM

In [ ]:
print(X_train.shape)

In [ ]:


# Aplatissement des images (conversion 28x28 en 784)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Normalisation des données (mise à l'échelle entre 0 et 1)
X_train = X_train / 255.0
X_test = X_test / 255.0
print(X_train.shape)
print(X_test.shape)

In [ ]:

#modèle SVM initial avec un noyau linéairean initial SVM model with linear kernel   
svm_linear = svm.SVC(kernel='linear')

# fit
svm_linear.fit(X_train, Y_train)

In [ ]:
predictions3 = svm_linear.predict(X_test)

images_to_display = X_test[:4]
predictions_to_display = predictions3[:4]

# Taille de la figure pour l'affichage
plt.figure(figsize=(10, 2))

for index, (image, prediction) in enumerate(zip(images_to_display, predictions_to_display)):
    plt.subplot(1, 4, index + 1)
    plt.imshow(image.reshape(28, 28), cmap='gray')
    plt.title(f"Prédiction: {prediction}")
    plt.axis('off')

plt.tight_layout()
plt.show()

# L'analyse de la performance du modèle SVM

In [ ]:
conf_matrix3 = metrics.confusion_matrix(y_true = Y_test, y_pred = predictions3)
conf_matrix3

In [ ]:
# Tracer la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix3, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
# measure accuracy
metrics.accuracy_score(y_true=Y_test, y_pred=predictions3)

In [ ]:
# class-wise accuracy
class_wise = metrics.classification_report(y_true=Y_test, y_pred=predictions3)
print(class_wise)

In [ ]:
!pip install nbconvert pypandoc
